As the second step of this tutorial, we will train an image model. This step can be run in parallel with Step 3 (training the text model).

This notebook was run on an AWS p3.2xlarge

# Tonks Image Model Training Pipeline

In [1]:
%load_ext autoreload

%autoreload 2

In [2]:
import sys
sys.path.append('../../')

In [3]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

Note: for images, we use the MultiInputMultiTaskLearner since we will send in the full image and a center crop of the image.

In [4]:
from tonks import MultiInputMultiTaskLearner, MultiDatasetLoader
from tonks.vision.dataset import TonksImageDataset
from tonks.vision.models import ResnetForMultiTaskClassification

## Load in train and validation datasets

First we load in the csv's we created in Step 1.
Remember to change the path if you stored your data somewhere other than the default.

In [5]:
TRAIN_COLOR_DF = pd.read_csv('data/color_swatches/color_train.csv')

In [6]:
VALID_COLOR_DF = pd.read_csv('data/color_swatches/color_valid.csv')

In [7]:
TRAIN_PATTERN_DF = pd.read_csv('data/pattern_swatches/pattern_train.csv')

In [8]:
VALID_PATTERN_DF = pd.read_csv('data/pattern_swatches/pattern_valid.csv')

You will most likely have to alter this to however big your batches can be on your machine

In [9]:
batch_size = 64

We use the `TonksImageDataSet` class to create train and valid datasets for each task.

Check out the documentation for infomation about the transformations.

In [10]:
color_train_dataset = TonksImageDataset(
    x=TRAIN_COLOR_DF['image_locs'],
    y=TRAIN_COLOR_DF['simple_color_cat'],
    transform='train',
    crop_transform='train'
)
color_valid_dataset = TonksImageDataset(
    x=VALID_COLOR_DF['image_locs'],
    y=VALID_COLOR_DF['simple_color_cat'],
    transform='val',
    crop_transform='val'
)

pattern_train_dataset = TonksImageDataset(
    x=TRAIN_PATTERN_DF['image_locs'],
    y=TRAIN_PATTERN_DF['pattern_type_cat'],
    transform='train',
    crop_transform='train'
)
pattern_valid_dataset = TonksImageDataset(
    x=VALID_PATTERN_DF['image_locs'],
    y=VALID_PATTERN_DF['pattern_type_cat'],
    transform='val',
    crop_transform='val'
)

We then put the datasets into a dictionary of dataloaders.

Each task is a key.

In [11]:
train_dataloaders_dict = {
    'color': DataLoader(color_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2),
    'pattern': DataLoader(pattern_train_dataset, batch_size=batch_size, shuffle=True, num_workers=2),
}
valid_dataloaders_dict = {
    'color': DataLoader(color_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8),
    'pattern': DataLoader(pattern_valid_dataset, batch_size=batch_size, shuffle=False, num_workers=8),
}

The dictionary of dataloaders is then put into an instance of the Tonks `MultiDatasetLoader` class.

In [12]:
TrainLoader = MultiDatasetLoader(loader_dict=train_dataloaders_dict)
len(TrainLoader)

8

In [13]:
ValidLoader = MultiDatasetLoader(loader_dict=valid_dataloaders_dict, shuffle=False)
len(ValidLoader)

3

We need to create a dictionary of the tasks and the number of unique values so that we can create our model. This is a `new_task_dict` because we are training new tasks from scratch, but we could potentially have a mix of new and pretrained tasks. See the Tonks documentation for more details.

In [14]:
new_task_dict = {
    'color': TRAIN_COLOR_DF['simple_color_cat'].nunique(),
    'pattern': TRAIN_PATTERN_DF['pattern_type_cat'].nunique(),
}

In [15]:
new_task_dict

{'color': 2, 'pattern': 2}

In [16]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


Create Model and Learner
===

These are completely new tasks so we use `new_task_dict`. If we had already trained a model on some tasks, we would use `pretrained_task_dict`.

And since these are new tasks, we set `load_pretrained_renset=True` to use the weights from Torch.

In [17]:
model = ResnetForMultiTaskClassification(
    new_task_dict=new_task_dict,
    load_pretrained_resnet=True
)

You will likely need to explore different values in this section to find some that work for your particular model.

In [18]:
loss_function = nn.CrossEntropyLoss()

lr_last = 1e-2
lr_main = 1e-4

optimizer = optim.Adam([
    {'params': model.resnet.parameters(), 'lr': lr_main},
    {'params': model.dense_layers.parameters(), 'lr': lr_last},
    {'params': model.new_classifiers.parameters(), 'lr': lr_last},
    
])

exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size= 4, gamma= 0.1)

In [19]:
learn = MultiInputMultiTaskLearner(model, TrainLoader, ValidLoader, new_task_dict)

Train model
===

As your model trains, you can see some output of how the model is performing overall and how it is doing on each individual task.

In [20]:
learn.fit(
    num_epochs=10,
    loss_function=loss_function,
    scheduler=exp_lr_scheduler,
    step_scheduler_on_batch=False,
    optimizer=optimizer,
    device=device,
    best_model=True
)

train_loss,val_loss,color_train_loss,color_val_loss,color_acc,pattern_train_loss,pattern_val_loss,pattern_acc,time
0.477539,1.950549,0.393019,0.956634,0.917431,0.842661,6.284015,0.680000,00:04
0.620837,1.322428,0.291730,0.273103,0.889908,2.042579,5.897488,0.440000,00:04
0.340648,1.139325,0.190167,0.651669,0.605505,0.990727,3.265508,0.480000,00:04
0.475544,0.343081,0.242980,0.210864,0.926606,1.480220,0.919548,0.840000,00:04
0.245107,0.381223,0.153973,0.215807,0.926606,0.638806,1.102434,0.720000,00:04
0.267711,0.218120,0.177266,0.158958,0.963303,0.658432,0.476063,0.720000,00:04
0.187381,0.293359,0.110614,0.254613,0.899083,0.519016,0.462289,0.760000,00:04
0.192692,0.281958,0.110441,0.223454,0.899083,0.548017,0.537036,0.840000,00:04
0.167040,0.391435,0.095955,0.333345,0.871560,0.474127,0.644704,0.680000,00:04
0.119068,0.386770,0.080910,0.332941,0.862385,0.283910,0.621464,0.760000,00:04


Epoch 5 best model saved with loss of 0.21811976695238655


Validate model
===

We provide a method on the learner called `get_val_preds`, which makes predictions on the validation data. You can then use this to analyze your model's performance in more detail.

In [21]:
pred_dict = learn.get_val_preds(device)

In [22]:
pred_dict

{'color': {'y_true': array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1.,
         0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0.,
         1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1.,
         1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
         1., 0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
         0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 0.,
         1., 1., 0., 1., 1., 1., 1.]),
  'y_pred': array([[2.45792523e-01, 7.54207492e-01],
         [2.71383077e-01, 7.28616953e-01],
         [2.12360248e-01, 7.87639737e-01],
         [9.95112598e-01, 4.88745281e-03],
         [1.20328076e-01, 8.79671931e-01],
         [2.19809279e-01, 7.80190766e-01],
         [1.63681567e-01, 8.36318374e-01],
         [1.44761562e-01, 8.55238438e-01],
         [1.51116133e-01, 8.48883808e-01],
         [1.44175127e-01, 8.55824888e-01],
         [9.76195037e-01, 2.38049980e-02

Save/Export Model
===

Once we are happy with our training we can save (or export) our model, using the `save` method (or `export`).

See the docs for the difference between `save` and `export`.

We will need the saved model later to use in the ensemble model

In [23]:
model.save(folder='models/', model_id='IMAGE_MODEL1')

In [24]:
model.export(folder='models/', model_id='IMAGE_MODEL1')

Now that we have an image model, we can move to `Step3_train_text_model`.